<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

yelp = pd.read_json('./data/review_sample.json', lines=True)

## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [3]:
import numpy as np
import pandas as pd
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

In [4]:
def tokenize(doc):
    
    return [token for token in simple_preprocess(doc) if token not in STOPWORDS]

In [5]:
yelp['tokens'] = yelp['text'].apply(lambda x: tokenize(x))
yelp['tokens'].head()

0    [beware, fake, fake, fake, small, business, lo...
1    [came, lunch, togo, service, quick, staff, fri...
2    [ve, vegas, dozens, times, stepped, foot, circ...
3    [went, night, closed, street, party, best, act...
4    [stars, bad, price, lunch, seniors, pay, eatin...
Name: tokens, dtype: object

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [45]:
# Instantiate vectorizer object
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english', min_df=0.025, max_df=0.95, ngram_range=(1,2))

# Create a vocabulary and get word counts per document
sparse = tfidf.fit_transform(yelp['text'])

# Convert sparse matrix to dense matrix and get column names
dtm = pd.DataFrame(sparse.todense(), columns=tfidf.get_feature_names())

dtm.head()

,able,absolutely,actually,ago,amazing,area,arrived,ask,asked,atmosphere,attentive,away,awesome,bad,bar,beautiful,beef,beer,best,better,big,bit,bread,breakfast,bring,brought,burger,business,busy,called,came,car,care,check,cheese,chicken,clean,close,coffee,cold,come,comes,coming,company,cooked,cool,couldn,couple,course,cream,customer,customer service,customers,cut,day,days,deal,decent,decided,definitely,delicious,dessert,different,dining,dinner,disappointed,dish,dishes,door,drink,drinks,drive,easy,eat,eating,end,enjoy,enjoyed,especially,excellent,expect,experience,extra,extremely,family,fantastic,far,fast,favorite,feel,felt,finally,fish,flavor,food,food good,free,fresh,fried,friend,friendly,friends,fries,fun,gave,getting,given,going,good,got,great,group,guy,guys,half,happy,hard,having,help,helpful,high,highly,highly recommend,home,hot,hotel,hour,hours,house,huge,husband,ice,inside,instead,isn,items,job,kids,kind,know,large,later,leave,left,let,like,line,little,live,ll,local,location,long,look,looked,looking,lot,lots,love,loved,lunch,makes,making,manager,maybe,meal,meat,menu,minutes,money,music,need,needed,needs,new,nice,night,offer,oh,ok,okay,old,open,options,order,ordered,outside,overall,owner,parking,pay,people,perfect,person,phone,pick,pizza,place,places,plus,pm,point,pork,pretty,price,prices,probably,problem,professional,quality,quick,quickly,reason,reasonable,recommend,restaurant,restaurants,return,review,reviews,rice,right,room,rude,said,salad,sandwich,sat,sauce,saw,second,selection,served,server,service,shop,sit,small,soup,special,spicy,spot,staff,star,stars,start,started,stay,steak,stop,store,strip,stuff,style,super,sure,sushi,sweet,table,tables,taste,tasted,tasty,tell,thank,thing,things,think,thought,time,times,today,told,took,town,tried,try,trying,use,usually,variety,ve,vegas,visit,wait,waited,waiting,waitress,walk,walked,want,wanted,wasn,water,way,week,went,wife,wine,wish,won,wonderful,work,working,worst,worth,wouldn,wrong,year,years,yelp
0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.240193,0.0,0.225422,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.261107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.258407,0.221494,0.0,0.134582,0.00000,0.0,0.260053,0.0,0.0,0.0,0.000000,0.000000,0.238601,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.210256,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.243832,0.515604,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.19776,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.204436,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.134424,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.240932,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.195655,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.300219,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [8]:
# Calculate distance of TF-IDF vectors using cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

dist_matrix = cosine_similarity(dtm)

In [9]:
dist = pd.DataFrame(dist_matrix)
dist.shape

(10000, 10000)

In [10]:
# Find top-10 most similar reviews (getting 11 since the first one will be the original document)
dist[0].nlargest(11)

0       1.000000
6899    0.528259
3714    0.524828
5465    0.470613
2131    0.470215
2865    0.460342
9036    0.438254
8470    0.430154
8531    0.427323
8411    0.413249
587     0.412669
Name: 0, dtype: float64

In [11]:
from sklearn.neighbors import NearestNeighbors

In [12]:
# Fit knn on TF-IDF vectors
nn = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=1.0)

In [57]:
# Create new yelp review
new_review = ['This store is the best store ever. It is way better than any of the other stores. I will only buy from this store for the rest of my life.']

In [14]:
# Transform new review
new_vect = tfidf.transform(new_review)

# Run model on new review
nn.kneighbors(new_vect.todense())

(array([[0.58203298, 0.67239107, 0.70284588, 0.70969724, 0.72370177,
         0.74111996, 0.74362168, 0.77708117, 0.78925934, 0.79180649]]),
 array([[ 305, 4558, 3837, 5791, 1592, 7666, 3545, 5705, 7990, 6626]],
       dtype=int64))

In [69]:
indicies = [305, 4558, 3837, 5791, 1592, 7666, 3545, 5705, 7990, 6626]
top_ten = new_review

for index in indicies:
    review = yelp['text'][index]
    print(review, '\n')

This store is did not have the Reese's peanut butter donut, I drove 15 miles for it. If your looking for it don't go to this store 

My favourite place to buy Kpop goods in Toronto. They have quite a wide selection of albums despite how small the store is and they've recently expanded their album section in the back of the store as well. Albums are fairly pricey (usually $25-$30) but it's worth it for the convenience instead of having to wait a month or more for something purchased from an online store.

The store also offers a wide array of pens and stationery and other cute little anime and pop culture trinkets.

Staff are always very friendly and talkative and make the store a very welcoming place. Gift wrapping is also available. 

I liked the variety of the maternity clothing here way better than the other store at the other premium outlets center/north. Also, the friendly ladies were very nice (smiling faces), and sincerely wanted to assist us with anything (but not in a pushy wa

## Insights

- The top-10 most similar reviews to the fake reviews all had to do with stores. They were a mix of good and bad reviews, but the common theme was that they mentioned the word store.

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [38]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
X = yelp['text']
y = yelp['stars']

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [46]:
# Create pipeline components
vect = TfidfVectorizer(tokenizer=tokenize, stop_words='english', min_df=0.025, max_df=0.95, ngram_range=(1,2))
clf = SGDClassifier(random_state=42)

# Define the pipeline
pipe = Pipeline([('vect', vect), ('clf', clf)])

# Define grid-search parameters
parameters = {
    'clf__loss': ['hinge', 'log'],
    'clf__alpha': [1e-4, 1e-3, 1e-2],
    'clf__tol': [1e-4, 1e-3, 1e-2]
}

In [47]:
grid_search = GridSearchCV(pipe, parameters, cv=2, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 2 folds for each of 18 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  1.3min finished


GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.95, max_features=None, min_df=0.025,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=T...dom_state=42, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'clf__loss': ['hinge', 'log'], 'clf__alpha': [0.0001, 0.001, 0.01], 'clf__tol': [0.0001, 0.001, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [48]:
grid_search.best_score_

0.58075

In [67]:
grid_search.best_estimator_.predict(new_review)

array([5], dtype=int64)

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [59]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

Learn the vocubalary of the yelp data:

In [60]:
id2word = corpora.Dictionary(yelp['tokens'])

Create a bag of words representation of the entire corpus

In [61]:
corpus = [id2word.doc2bow(text) for text in yelp['tokens']]

Your LDA model should be ready for estimation: 

In [62]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 10 # You can change this parameter
                  )

Create 1-2 visualizations of the results

In [63]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [64]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

C:\Users\theta\Anaconda3\envs\u4_s1_nlp\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.003886 -0.004481       1        1  10.649356
1     -0.004397  0.001927       2        1  10.531096
9     -0.001693  0.001880       3        1  10.516980
0      0.001279 -0.001309       4        1  10.190803
2      0.000984 -0.002258       5        1  10.036341
8     -0.001313  0.004798       6        1   9.859295
6     -0.000457 -0.001812       7        1   9.741371
3      0.002902 -0.000740       8        1   9.550577
4      0.002441  0.001474       9        1   9.471384
5      0.004140  0.000522      10        1   9.452796, topic_info=      Category         Freq            Term        Total  loglift  logprob
40     Default  4455.000000            good  4455.000000  30.0000  30.0000
144    Default  4755.000000            food  4755.000000  29.0000  29.0000
208    Default  4130.000000           great  4130.000000  28.0000  28.0000
351    Default  1401.000000            come  1401.000000  27.0000  27.0000
46     Default  1506.000000           staff  1506.000000  26.0000  26.0000
18     Default  3329.000000            like  3329.000000  25.0000  25.0000
133    Default  4557.000000           place  4557.000000  24.0000  24.0000
45     Default  3503.000000         service  3503.000000  23.0000  23.0000
387    Default  1450.000000           order  1450.000000  22.0000  22.0000
192    Default  1312.000000         amazing  1312.000000  21.0000  21.0000
371    Default   906.000000           fresh   906.000000  20.0000  20.0000
92     Default  1688.000000            nice  1688.000000  19.0000  19.0000
137    Default   982.000000             bar   982.000000  18.0000  18.0000
381    Default  1373.000000          little  1373.000000  17.0000  17.0000
75     Default  1987.000000             got  1987.000000  16.0000  16.0000
122    Default  1750.000000            best  1750.000000  15.0000  15.0000
94     Default   940.000000           night   940.000000  14.0000  14.0000
735    Default  1586.000000            love  1586.000000  13.0000  13.0000
812    Default   674.000000           store   674.000000  12.0000  12.0000
426    Default  1116.000000          better  1116.000000  11.0000  11.0000
165    Default   843.000000           think   843.000000  10.0000  10.0000
34     Default  3303.000000            time  3303.000000   9.0000   9.0000
394    Default   763.000000           pizza   763.000000   8.0000   8.0000
400    Default   789.000000           sauce   789.000000   7.0000   7.0000
359    Default  1194.000000       delicious  1194.000000   6.0000   6.0000
180    Default  1193.000000            menu  1193.000000   5.0000   5.0000
940    Default   697.000000           happy   697.000000   4.0000   4.0000
358    Default  1394.000000      definitely  1394.000000   3.0000   3.0000
361    Default   668.000000          dinner   668.000000   2.0000   2.0000
478    Default  1175.000000            know  1175.000000   1.0000   1.0000
25544   Topic1     1.062083         booster     3.091227   1.1713 -10.7827
20436   Topic1     0.525175          kristy     1.618867   1.1139 -11.4870
26439   Topic1     0.965702           dolci     3.091250   1.0762 -10.8778
16261   Topic1     1.124059          harlow     3.745084   1.0362 -10.7260
20435   Topic1     0.469595         jeffery     1.617739   1.0028 -11.5988
16864   Topic1     0.444240              yc     1.616274   0.9482 -11.6543
26362   Topic1     0.846186        organisé     3.092925   0.9435 -11.0099
24986   Topic1     0.834198             cos     3.089340   0.9304 -11.0242
25640   Topic1     0.821776     walkthrough     3.089363   0.9154 -11.0392
16670   Topic1     0.590116         bregman     2.278034   0.8889 -11.3704
13754   Topic1     0.415873             pnw     1.617852   0.8812 -11.7203
13120   Topic1     1.125798      greenhouse     4.410706   0.8741 -10.7244
26365   Topic1     0.785810     respectueux     3.092986   0.8695 -11.0840
17593   T

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)